<a href="https://colab.research.google.com/github/oxanaRC/CE902/blob/main/Building_experimental_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

14.08.21

1. Extracting property sales dataset from HarperDB
2. Extracting 'number of bedrooms' data from HarperDB and joining to the property sales dataset.
3. Extracting sales-postcodes-mappings-to-schools-postcodes from HarperDB
4. Constructing features related to the schools mapping data (num_of_sec_sch,	score_disadv,	closest_disadv_prct,	closest_aver_attain_eng,	closest_aver_attain_math,	closest_sch_distance) and merging schools related features to the main dataset.
5. Feature to represent a historic point: e.g. 01-2018=>1, 02-2018=>2 etc
6. Extracting historic (3 months and 4 months) median price per bedroom for each postal area and merging this to the main dataset.
7.Calculate each price per bedroom for comparing with historic median ppd.
8. Extracting data on distances from each sales postcode to three locations (in London, Birmingham, Manchester) from HarperDB and merging this with the main dataset.
9. Extracting lat and long for each postcode from HarperDB and merging this with the main dataset




Output three csv files:
1) dataset for training and testing ()
2) dataset for validation
3) dataset for records identification ('location key', 'postal code' and 'district) - this to be joined with data after predictions were done so the quality of predictions could be analysed by location.



In [2]:
!pip install harperdb

In [3]:
import harperdb
import pandas as pd
import numpy as np
from pathlib import Path

In [4]:
url='https://ce901data-orc0770.harperdbcloud.com'
db=harperdb.HarperDB(url=url,
            username='myuser',
            password='abcdef',
            timeout=500)

In [5]:
db

Below data is extracted from the SQL database, however because of poor performance of the JOIN SQL operations, instead of relational SQL statements, the JOIN operations are performed in pandas DataFrames (according to https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html).


The data extraction from SQL tables in placed within the loop to avoid the database timeout issues.

This extracts property sales data.



In [6]:
list_of_dfs = {}
count=0
for mt in range(1, 11):
  for yr in range (2018,2021):
    year=yr
    month=mt
    count=count+1
    sqlstring='SELECT propertypurchases.unique_id, propertypurchases.price_paid, propertypurchases.town, propertypurchases.district, propertypurchases.estate_type, propertypurchases.property_type, propertypurchases.deed_date, propertypurchases.postcode FROM projectdata.propertypurchases WHERE deed_date LIKE "%{}/{}"'
    sqlstring=sqlstring.format(month,year)
    #print(sqlstring)
    my_db_data=db.sql(sqlstring)
    list_of_dfs[count] = pd.DataFrame(my_db_data)
    print(list_of_dfs[count].shape)
df_prop_sales = pd.concat(list_of_dfs, sort=False)
#excluding 'other' property types (i.e. commercial) from the properties sales data 
df_prop_sales=df_prop_sales[(df_prop_sales['property_type']!='O')]
df_prop_sales

(169065, 8)
(159110, 8)
(133520, 8)
(157760, 8)
(156576, 8)
(94872, 8)
(84147, 8)
(81931, 8)
(71124, 8)
(72140, 8)
(74714, 8)
(29808, 8)
(81439, 8)
(81425, 8)
(36483, 8)
(95768, 8)
(85103, 8)
(53814, 8)
(88779, 8)
(89006, 8)
(62290, 8)
(101675, 8)
(93414, 8)
(62584, 8)
(83257, 8)
(82914, 8)
(69959, 8)
(93078, 8)
(91044, 8)
(84980, 8)


unique_id  price_paid  ...   deed_date postcode
1  0      64342BFD-ABA7-422C-E053-6C04A8C0FB8A     1740000  ...  11/01/2018   E8 4NE
   1      64342BFD-ABAA-422C-E053-6C04A8C0FB8A     1525000  ...  19/01/2018   N5 1BZ
   2      64342BFD-ABAB-422C-E053-6C04A8C0FB8A     3306000  ...  17/01/2018  W11 1NZ
   3      64342BFD-ABAC-422C-E053-6C04A8C0FB8A      442000  ...  04/01/2018  IG3 9NB
   4      64342BFD-ABB0-422C-E053-6C04A8C0FB8A     1375000  ...  12/01/2018  W12 8JN
...                                        ...         ...  ...         ...      ...
30 84973  C18F412B-BE2B-81A6-E053-6B04A8C0AD18      125000  ...  09/10/2020  LS9 9DA
   84974  C18F412B-BEA1-81A6-E053-6B04A8C0AD18      147500  ...  06/10/2020  HD5 9NY
   84975  C18F412B-BEA2-81A6-E053-6B04A8C0AD18      145000  ...  06/10/2020  HD5 9NY
   84977  C18F412B-BEA5-81A6-E053-6B04A8C0AD18      104000  ...  02/10/2020  WF4 6AB
   84979  C18F412B-BEE8-81A6-E053-6B04A8C0AD18      169000  ...  30/10/2020  WF7 5PG

[2563547 rows x 8 columns]

Below extracting number of bedrooms data for each property sale id.


In [7]:
sqlstring='SELECT bedrooms.id, bedrooms.num_of_beds FROM projectdata.bedrooms WHERE num_of_beds>0'
bedrooms=db.sql(sqlstring)
df_bedrooms=pd.DataFrame(bedrooms)
df_bedrooms.rename(columns = {'id':'unique_id'}, inplace = True)
df_bedrooms

,unique_id,num_of_beds
0,726BF13A-B9B6-0A46-E053-6C04A8C01D0D,1
1,9B361206-34DE-1904-E053-6B04A8C0EEB5,1
2,726BF13B-13CE-0A46-E053-6C04A8C01D0D,1
3,A71375FD-EBA1-7576-E053-6C04A8C0462F,1
4,8355F008-F016-55C5-E053-6B04A8C0D090,1
...,...,...
1231571,8355F009-2B4F-55C5-E053-6B04A8C0D090,47
1231572,8A78B2B0-784A-5CB0-E053-6B04A8C0F504,48
1231573,8355F009-8A15-55C5-E053-6B04A8C0D090,55
1231574,8A78B2B0-4326-5CB0-E053-6B04A8C0F504,81


In [8]:
#merging bedrooms data with property sales data.
data = pd.merge(df_bedrooms, df_prop_sales, how="left", on="unique_id")
data

,unique_id,num_of_beds,price_paid,town,district,estate_type,property_type,deed_date,postcode
0,726BF13A-B9B6-0A46-E053-6C04A8C01D0D,1,100000.0,MANCHESTER,SALFORD,L,T,20/07/2018,M30 8EG
1,9B361206-34DE-1904-E053-6B04A8C0EEB5,1,79500.0,MANCHESTER,BURY,L,S,08/11/2019,M25 1ET
2,726BF13B-13CE-0A46-E053-6C04A8C01D0D,1,128000.0,NORWICH,NORWICH,L,F,20/04/2018,NR3 1SX
3,A71375FD-EBA1-7576-E053-6C04A8C0462F,1,314000.0,LONDON,LAMBETH,L,F,21/02/2020,SW12 0JP
4,8355F008-F016-55C5-E053-6B04A8C0D090,1,170000.0,BRAINTREE,BRAINTREE,L,F,01/02/2019,CM7 1TD
...,...,...,...,...,...,...,...,...,...
1231571,8355F009-2B4F-55C5-E053-6B04A8C0D090,47,79000.0,LEIGH,WIGAN,L,T,08/02/2019,WN7 1NG
1231572,8A78B2B0-784A-5CB0-E053-6B04A8C0F504,48,53000.0,BLACKWOOD,CAERPHILLY,F,S,12/04/2019,NP12 3LQ
1231573,8355F009-8A15-55C5-E053-6B04A8C0D090,55,70000.0,FLINT,FLINTSHIRE,F,T,11/02/2019,CH6 5DY
1231574,8A78B2B0-4326-5CB0-E053-6B04A8C0F504,81,95000.0,TODMORDEN,CALDERDALE,L,T,08/04/2019,OL14 8NL


In [9]:
#check rows where postcode is NAN - these are the result of rows added on the Join with bedrooms data where bedrooms data is for sale happened after the considered period
#data_t=data[ (data['postcode'].isnull()) ]
#drop NAN rows from data
data=data.dropna(subset=['postcode'])     
data 

,unique_id,num_of_beds,price_paid,town,district,estate_type,property_type,deed_date,postcode
0,726BF13A-B9B6-0A46-E053-6C04A8C01D0D,1,100000.0,MANCHESTER,SALFORD,L,T,20/07/2018,M30 8EG
1,9B361206-34DE-1904-E053-6B04A8C0EEB5,1,79500.0,MANCHESTER,BURY,L,S,08/11/2019,M25 1ET
2,726BF13B-13CE-0A46-E053-6C04A8C01D0D,1,128000.0,NORWICH,NORWICH,L,F,20/04/2018,NR3 1SX
3,A71375FD-EBA1-7576-E053-6C04A8C0462F,1,314000.0,LONDON,LAMBETH,L,F,21/02/2020,SW12 0JP
4,8355F008-F016-55C5-E053-6B04A8C0D090,1,170000.0,BRAINTREE,BRAINTREE,L,F,01/02/2019,CM7 1TD
...,...,...,...,...,...,...,...,...,...
1231571,8355F009-2B4F-55C5-E053-6B04A8C0D090,47,79000.0,LEIGH,WIGAN,L,T,08/02/2019,WN7 1NG
1231572,8A78B2B0-784A-5CB0-E053-6B04A8C0F504,48,53000.0,BLACKWOOD,CAERPHILLY,F,S,12/04/2019,NP12 3LQ
1231573,8355F009-8A15-55C5-E053-6B04A8C0D090,55,70000.0,FLINT,FLINTSHIRE,F,T,11/02/2019,CH6 5DY
1231574,8A78B2B0-4326-5CB0-E053-6B04A8C0F504,81,95000.0,TODMORDEN,CALDERDALE,L,T,08/04/2019,OL14 8NL


In [10]:
#removing duplicates (some bedroom data was loaded to the HarperDB twice by mistake)
data=data.drop_duplicates()
data

,unique_id,num_of_beds,price_paid,town,district,estate_type,property_type,deed_date,postcode
0,726BF13A-B9B6-0A46-E053-6C04A8C01D0D,1,100000.0,MANCHESTER,SALFORD,L,T,20/07/2018,M30 8EG
1,9B361206-34DE-1904-E053-6B04A8C0EEB5,1,79500.0,MANCHESTER,BURY,L,S,08/11/2019,M25 1ET
2,726BF13B-13CE-0A46-E053-6C04A8C01D0D,1,128000.0,NORWICH,NORWICH,L,F,20/04/2018,NR3 1SX
3,A71375FD-EBA1-7576-E053-6C04A8C0462F,1,314000.0,LONDON,LAMBETH,L,F,21/02/2020,SW12 0JP
4,8355F008-F016-55C5-E053-6B04A8C0D090,1,170000.0,BRAINTREE,BRAINTREE,L,F,01/02/2019,CM7 1TD
...,...,...,...,...,...,...,...,...,...
1231571,8355F009-2B4F-55C5-E053-6B04A8C0D090,47,79000.0,LEIGH,WIGAN,L,T,08/02/2019,WN7 1NG
1231572,8A78B2B0-784A-5CB0-E053-6B04A8C0F504,48,53000.0,BLACKWOOD,CAERPHILLY,F,S,12/04/2019,NP12 3LQ
1231573,8355F009-8A15-55C5-E053-6B04A8C0D090,55,70000.0,FLINT,FLINTSHIRE,F,T,11/02/2019,CH6 5DY
1231574,8A78B2B0-4326-5CB0-E053-6B04A8C0F504,81,95000.0,TODMORDEN,CALDERDALE,L,T,08/04/2019,OL14 8NL


In [6]:
#Extracting sales-postcodes-mappings-to-schools-postcodes 
#sales-postcodes-mappings-to-schools-postcodes data is kept in 7 separate SQL tables (due to processing issues with HarperDB)
#Extraction of the data is done from the 7 seprate SQL tables and then the data is merged in one df_local_schools dataframe
list_of_mapping_dfs = {}
sqlstring='SELECT map_sales_pc_to_school_a_c.sales_postcode, map_sales_pc_to_school_a_c.urn, map_sales_pc_to_school_a_c.distance, map_sales_pc_to_school_a_c.disadv_prct, map_sales_pc_to_school_a_c.aver_attain_eng , map_sales_pc_to_school_a_c.aver_attain_math FROM projectdata.map_sales_pc_to_school_a_c'
school_pc_mapping=db.sql(sqlstring)
list_of_mapping_dfs[1]=pd.DataFrame(school_pc_mapping)
sqlstring='SELECT map_sales_pc_to_school_d_i.sales_postcode, map_sales_pc_to_school_d_i.urn, map_sales_pc_to_school_d_i.distance, map_sales_pc_to_school_d_i.disadv_prct, map_sales_pc_to_school_d_i.aver_attain_eng , map_sales_pc_to_school_d_i.aver_attain_math FROM projectdata.map_sales_pc_to_school_d_i'
school_pc_mapping=db.sql(sqlstring)
list_of_mapping_dfs[2]=pd.DataFrame(school_pc_mapping)
sqlstring='SELECT map_sales_pc_to_school_j_m.sales_postcode, map_sales_pc_to_school_j_m.urn, map_sales_pc_to_school_j_m.distance, map_sales_pc_to_school_j_m.disadv_prct, map_sales_pc_to_school_j_m.aver_attain_eng , map_sales_pc_to_school_j_m.aver_attain_math FROM projectdata.map_sales_pc_to_school_j_m'
school_pc_mapping=db.sql(sqlstring)
list_of_mapping_dfs[3]=pd.DataFrame(school_pc_mapping)
sqlstring='SELECT map_sales_pc_to_school_n_q.sales_postcode, map_sales_pc_to_school_n_q.urn, map_sales_pc_to_school_n_q.distance, map_sales_pc_to_school_n_q.disadv_prct, map_sales_pc_to_school_n_q.aver_attain_eng , map_sales_pc_to_school_n_q.aver_attain_math FROM projectdata.map_sales_pc_to_school_n_q'
school_pc_mapping=db.sql(sqlstring)
list_of_mapping_dfs[4]=pd.DataFrame(school_pc_mapping)
sqlstring='SELECT map_sales_pc_to_school_r.sales_postcode, map_sales_pc_to_school_r.urn, map_sales_pc_to_school_r.distance, map_sales_pc_to_school_r.disadv_prct, map_sales_pc_to_school_r.aver_attain_eng , map_sales_pc_to_school_r.aver_attain_math FROM projectdata.map_sales_pc_to_school_r'
school_pc_mapping=db.sql(sqlstring)
list_of_mapping_dfs[5]=pd.DataFrame(school_pc_mapping)
sqlstring='SELECT map_sales_pc_to_school_s.sales_postcode, map_sales_pc_to_school_s.urn, map_sales_pc_to_school_s.distance, map_sales_pc_to_school_s.disadv_prct, map_sales_pc_to_school_s.aver_attain_eng , map_sales_pc_to_school_s.aver_attain_math FROM projectdata.map_sales_pc_to_school_s'
school_pc_mapping=db.sql(sqlstring)
list_of_mapping_dfs[6]=pd.DataFrame(school_pc_mapping)
sqlstring='SELECT map_sales_pc_to_school_t_z.sales_postcode, map_sales_pc_to_school_t_z.urn, map_sales_pc_to_school_t_z.distance, map_sales_pc_to_school_t_z.disadv_prct, map_sales_pc_to_school_t_z.aver_attain_eng , map_sales_pc_to_school_t_z.aver_attain_math FROM projectdata.map_sales_pc_to_school_t_z'
school_pc_mapping=db.sql(sqlstring)
list_of_mapping_dfs[7]=pd.DataFrame(school_pc_mapping)
df_local_schools = pd.concat(list_of_mapping_dfs, sort=False)
df_local_schools

sales_postcode     urn  ...  aver_attain_eng aver_attain_math
1 0             AL1 1AJ  138042  ...              9.4              8.3
  1             AL1 1AJ  138106  ...             12.3             10.8
  2             AL1 1AJ  137038  ...             11.2             11.1
  3             AL1 1AJ  138582  ...             10.6              9.2
  4             AL1 1AJ  117674  ...              2.4              2.3
...                 ...     ...  ...              ...              ...
7 772678        YO8 9XP  144929  ...              9.7              8.3
  772679        YO8 9XP  143138  ...              9.6             10.1
  772680        YO8 9YB  121702  ...              9.4              9.0
  772681        YO8 9YB  143138  ...              9.6             10.1
  772682        YO8 9YE  143138  ...              9.6             10.1

[6520480 rows x 6 columns]

In [7]:
#remove special schools 
df_local_schools.drop(df_local_schools[df_local_schools['disadv_prct']=='NP'].index, inplace = True)
df_local_schools

sales_postcode     urn  ...  aver_attain_eng aver_attain_math
1 0             AL1 1AJ  138042  ...              9.4              8.3
  1             AL1 1AJ  138106  ...             12.3             10.8
  2             AL1 1AJ  137038  ...             11.2             11.1
  3             AL1 1AJ  138582  ...             10.6              9.2
  4             AL1 1AJ  117674  ...              2.4              2.3
...                 ...     ...  ...              ...              ...
7 772678        YO8 9XP  144929  ...              9.7              8.3
  772679        YO8 9XP  143138  ...              9.6             10.1
  772680        YO8 9YB  121702  ...              9.4              9.0
  772681        YO8 9YB  143138  ...              9.6             10.1
  772682        YO8 9YE  143138  ...              9.6             10.1

[6495209 rows x 6 columns]

In [8]:
#cheking if the school mappings data contains Nan
data_test = df_local_schools[(df_local_schools['disadv_prct'].isnull())]
data_test

,,sales_postcode,urn,distance,disadv_prct,aver_attain_eng,aver_attain_math


In [9]:
#cheking if the school mappings data contains duplicates
school_map_dupl = df_local_schools[df_local_schools.duplicated()]
school_map_dupl

,,sales_postcode,urn,distance,disadv_prct,aver_attain_eng,aver_attain_math


In [10]:
#adding 'weights' to the school mapping data: calculated as 6km minus the distance from the sale postcode to a school
#the closer a school to the postocode, the greater the weight of that school
df_local_schools['weight']=6-df_local_schools['distance']
df_local_schools['disadv_prct']=df_local_schools['disadv_prct'].str.replace('%','')
df_local_schools['disadv_prct'] = df_local_schools['disadv_prct'].replace(np.nan, 0)  #fillna(0)
df_local_schools['disadv_prct'] = df_local_schools['disadv_prct']. fillna(0)
df_local_schools['disadv_prct']= pd.to_numeric(df_local_schools['disadv_prct'], downcast='float')
df_local_schools

sales_postcode     urn  ...  aver_attain_math    weight
1 0             AL1 1AJ  138042  ...               8.3  4.422326
  1             AL1 1AJ  138106  ...              10.8  5.067710
  2             AL1 1AJ  137038  ...              11.1  4.512236
  3             AL1 1AJ  138582  ...               9.2  4.636620
  4             AL1 1AJ  117674  ...               2.3  3.589736
...                 ...     ...  ...               ...       ...
7 772678        YO8 9XP  144929  ...               8.3  1.906942
  772679        YO8 9XP  143138  ...              10.1  5.299211
  772680        YO8 9YB  121702  ...               9.0  0.579519
  772681        YO8 9YB  143138  ...              10.1  0.917180
  772682        YO8 9YE  143138  ...              10.1  0.188667

[6495209 rows x 7 columns]

In [11]:
#calculating a score for each school mapping
df_local_schools['score_disadv']=df_local_schools['weight'].astype(float)*df_local_schools['disadv_prct'].astype(float)
df_local_schools

sales_postcode     urn  ...    weight  score_disadv
1 0             AL1 1AJ  138042  ...  4.422326    110.558155
  1             AL1 1AJ  138106  ...  5.067710     50.677096
  2             AL1 1AJ  137038  ...  4.512236     45.122361
  3             AL1 1AJ  138582  ...  4.636620    115.915492
  4             AL1 1AJ  117674  ...  3.589736    283.589142
...                 ...     ...  ...       ...           ...
7 772678        YO8 9XP  144929  ...  1.906942     36.231893
  772679        YO8 9XP  143138  ...  5.299211     95.385806
  772680        YO8 9YB  121702  ...  0.579519      9.851820
  772681        YO8 9YB  143138  ...  0.917180     16.509242
  772682        YO8 9YE  143138  ...  0.188667      3.395998

[6495209 rows x 8 columns]

In [12]:
#testing:
#there are rows where urn is zero (it means no school, however this row has been counted at 1 school)
df_test1=df_local_schools[(df_local_schools['sales_postcode'] =='IP30 0AG') ]
df_test1

,,sales_postcode,urn,distance,disadv_prct,aver_attain_eng,aver_attain_math,weight,score_disadv
2,1048362,IP30 0AG,0,0.0,0.0,0.0,0.0,6.0,0.0


In [17]:
#counting number of secondary schools (in the radius of 6 km) for each sales postcode
g_sch_num = df_local_schools.groupby(['sales_postcode'])['urn'].count().reset_index()
g_sch_num.rename(columns = {'urn':'num_of_sec_sch'}, inplace = True)
g_sch_num.rename(columns = {'sales_postcode':'postcode'}, inplace = True)
g_sch_num

,postcode,num_of_sec_sch
0,AL1 1AJ,10
1,AL1 1AR,10
2,AL1 1AT,10
3,AL1 1AU,10
4,AL1 1DG,11
...,...,...
580840,YO8 9XL,3
580841,YO8 9XN,3
580842,YO8 9XP,3
580843,YO8 9YB,2


In [18]:
#calculating average weighted disability score for each postcode
g_aver_score_disadv = df_local_schools.groupby(['sales_postcode'])['score_disadv'].mean().reset_index()
g_aver_score_disadv.rename(columns = {'sales_postcode':'postcode'}, inplace = True)
g_aver_score_disadv

,postcode,score_disadv
0,AL1 1AJ,78.575432
1,AL1 1AR,70.273088
2,AL1 1AT,73.883530
3,AL1 1AU,73.144251
4,AL1 1DG,72.342235
...,...,...
580840,YO8 9XL,70.072425
580841,YO8 9XN,69.052363
580842,YO8 9XP,72.104548
580843,YO8 9YB,13.180531


In [19]:
#For calculating English and Math attainment score for each postcode we'll only consider the most closest school 
#(as taking average score value for all local schools will not be helpful - for example if one closest school in a rural area is 5.9 km away from a property - the distance weighting will diminish the score value)
#extracting the closest schools:
g_closest_schools = df_local_schools.groupby(['sales_postcode'])['distance'].min().reset_index()
g_closest_schools.rename(columns = {'distance':'closest_sch_distance'}, inplace = True)
g_closest_schools

,sales_postcode,closest_sch_distance
0,AL1 1AJ,0.932290
1,AL1 1AR,0.455425
2,AL1 1AT,0.676792
3,AL1 1AU,0.638617
4,AL1 1DG,1.093564
...,...,...
580840,YO8 9XL,0.690030
580841,YO8 9XN,0.742595
580842,YO8 9XP,0.700789
580843,YO8 9YB,5.082820


In [20]:
#joining the closest school table with df_local_schools to obtains english and math attainment for the closest schools
g_closest_schools_with_data = pd.merge(df_local_schools, g_closest_schools, how="left", on="sales_postcode")
g_closest_schools_with_data.drop(g_closest_schools_with_data[g_closest_schools_with_data['distance']!=g_closest_schools_with_data['closest_sch_distance']].index, inplace = True)
g_closest_schools_with_data.rename(columns = {'disadv_prct':'closest_disadv_prct'}, inplace = True)
g_closest_schools_with_data.rename(columns = {'sales_postcode':'postcode'}, inplace = True)
g_closest_schools_with_data.rename(columns = {'aver_attain_eng':'closest_aver_attain_eng'}, inplace = True)
g_closest_schools_with_data.rename(columns = {'aver_attain_math':'closest_aver_attain_math'}, inplace = True)
g_closest_schools_with_data=g_closest_schools_with_data.drop(['distance', 'weight', 'score_disadv'], axis=1)
g_closest_schools_with_data

,postcode,urn,closest_disadv_prct,closest_aver_attain_eng,closest_aver_attain_math,closest_sch_distance
1,AL1 1AJ,138106,10.0,12.3,10.8,0.932290
13,AL1 1AR,138582,25.0,10.6,9.2,0.455425
23,AL1 1AT,138582,25.0,10.6,9.2,0.676792
33,AL1 1AU,138582,25.0,10.6,9.2,0.638617
40,AL1 1DG,138042,25.0,9.4,8.3,1.093564
...,...,...,...,...,...,...
6495199,YO8 9XL,143138,18.0,9.6,10.1,0.690030
6495202,YO8 9XN,143138,18.0,9.6,10.1,0.742595
6495205,YO8 9XP,143138,18.0,9.6,10.1,0.700789
6495207,YO8 9YB,143138,18.0,9.6,10.1,5.082820


In [21]:
#some postcodes will have more than one school at exactly the same closest distance 
data_test2 = g_closest_schools_with_data.groupby(['postcode'])['urn'].count().reset_index()
data_test3=data_test2[(data_test2['urn'] !=1)]
data_test3

,postcode,urn
3342,B13 8PG,2
3343,B13 8PP,2
3344,B13 8PQ,2
3733,B14 6AB,2
3736,B14 6AH,2
...,...,...
573712,WV6 8PH,2
573713,WV6 8PJ,2
573715,WV6 8PP,2
573716,WV6 8PQ,2


In [22]:
#chosing maximum closest_disadv_prct,	closest_aver_attain_eng,	closest_aver_attain_math for each sales postcode (i.e. if there are more than one school at exactly the same distance, one value is chosen)
g_closest_schools_max1 = g_closest_schools_with_data.groupby(['postcode'])['closest_disadv_prct'].max().reset_index()
g_closest_schools_max2 = g_closest_schools_with_data.groupby(['postcode'])['closest_aver_attain_eng'].max().reset_index()
g_closest_schools_max3 = g_closest_schools_with_data.groupby(['postcode'])['closest_aver_attain_math'].max().reset_index()
g_closest_schools_max4 = g_closest_schools_with_data.groupby(['postcode'])['closest_sch_distance'].max().reset_index()
g_closest_schools_with_data_max=pd.merge(g_closest_schools_max1,g_closest_schools_max2, how="left", on="postcode")
g_closest_schools_with_data_max=pd.merge(g_closest_schools_with_data_max,g_closest_schools_max3, how="left", on="postcode")
g_closest_schools_with_data_max=pd.merge(g_closest_schools_with_data_max,g_closest_schools_max4, how="left", on="postcode")
g_closest_schools_with_data_max

,postcode,closest_disadv_prct,closest_aver_attain_eng,closest_aver_attain_math,closest_sch_distance
0,AL1 1AJ,10.0,12.3,10.8,0.932290
1,AL1 1AR,25.0,10.6,9.2,0.455425
2,AL1 1AT,25.0,10.6,9.2,0.676792
3,AL1 1AU,25.0,10.6,9.2,0.638617
4,AL1 1DG,25.0,9.4,8.3,1.093564
...,...,...,...,...,...
580840,YO8 9XL,18.0,9.6,10.1,0.690030
580841,YO8 9XN,18.0,9.6,10.1,0.742595
580842,YO8 9XP,18.0,9.6,10.1,0.700789
580843,YO8 9YB,18.0,9.6,10.1,5.082820


In [23]:
#join all school data together
#for each prop sales postcode we have number of secondary schools in radius 6 km, weighted average disadvantage score, closest school disadv %, closest sch eng and math attaint, closest sch distance
#alltogether 6 features related to schools data
prepared_sch_data = pd.merge(g_sch_num, g_aver_score_disadv, how="left", on="postcode")
prepared_sch_data = pd.merge(prepared_sch_data, g_closest_schools_with_data_max, how="left", on="postcode")
prepared_sch_data

,postcode,num_of_sec_sch,score_disadv,closest_disadv_prct,closest_aver_attain_eng,closest_aver_attain_math,closest_sch_distance
0,AL1 1AJ,10,78.575432,10.0,12.3,10.8,0.932290
1,AL1 1AR,10,70.273088,25.0,10.6,9.2,0.455425
2,AL1 1AT,10,73.883530,25.0,10.6,9.2,0.676792
3,AL1 1AU,10,73.144251,25.0,10.6,9.2,0.638617
4,AL1 1DG,11,72.342235,25.0,9.4,8.3,1.093564
...,...,...,...,...,...,...,...
580840,YO8 9XL,3,70.072425,18.0,9.6,10.1,0.690030
580841,YO8 9XN,3,69.052363,18.0,9.6,10.1,0.742595
580842,YO8 9XP,3,72.104548,18.0,9.6,10.1,0.700789
580843,YO8 9YB,2,13.180531,18.0,9.6,10.1,5.082820


In [24]:
data

,unique_id,num_of_beds,price_paid,town,district,estate_type,property_type,deed_date,postcode
0,726BF13A-B9B6-0A46-E053-6C04A8C01D0D,1,100000.0,MANCHESTER,SALFORD,L,T,20/07/2018,M30 8EG
1,9B361206-34DE-1904-E053-6B04A8C0EEB5,1,79500.0,MANCHESTER,BURY,L,S,08/11/2019,M25 1ET
2,726BF13B-13CE-0A46-E053-6C04A8C01D0D,1,128000.0,NORWICH,NORWICH,L,F,20/04/2018,NR3 1SX
3,A71375FD-EBA1-7576-E053-6C04A8C0462F,1,314000.0,LONDON,LAMBETH,L,F,21/02/2020,SW12 0JP
4,8355F008-F016-55C5-E053-6B04A8C0D090,1,170000.0,BRAINTREE,BRAINTREE,L,F,01/02/2019,CM7 1TD
...,...,...,...,...,...,...,...,...,...
1231571,8355F009-2B4F-55C5-E053-6B04A8C0D090,47,79000.0,LEIGH,WIGAN,L,T,08/02/2019,WN7 1NG
1231572,8A78B2B0-784A-5CB0-E053-6B04A8C0F504,48,53000.0,BLACKWOOD,CAERPHILLY,F,S,12/04/2019,NP12 3LQ
1231573,8355F009-8A15-55C5-E053-6B04A8C0D090,55,70000.0,FLINT,FLINTSHIRE,F,T,11/02/2019,CH6 5DY
1231574,8A78B2B0-4326-5CB0-E053-6B04A8C0F504,81,95000.0,TODMORDEN,CALDERDALE,L,T,08/04/2019,OL14 8NL


In [25]:
#append prepared_sch_data to data
data = pd.merge(data, prepared_sch_data, how="left", on="postcode")
data

,unique_id,num_of_beds,price_paid,town,district,estate_type,property_type,deed_date,postcode,num_of_sec_sch,score_disadv,closest_disadv_prct,closest_aver_attain_eng,closest_aver_attain_math,closest_sch_distance
0,726BF13A-B9B6-0A46-E053-6C04A8C01D0D,1,100000.0,MANCHESTER,SALFORD,L,T,20/07/2018,M30 8EG,23.0,110.178111,96.0,0.5,0.9,0.671216
1,9B361206-34DE-1904-E053-6B04A8C0EEB5,1,79500.0,MANCHESTER,BURY,L,S,08/11/2019,M25 1ET,24.0,78.275669,15.0,10.8,9.4,0.320619
2,726BF13B-13CE-0A46-E053-6C04A8C01D0D,1,128000.0,NORWICH,NORWICH,L,F,20/04/2018,NR3 1SX,12.0,85.979811,20.0,11.5,9.8,0.584296
3,A71375FD-EBA1-7576-E053-6C04A8C0462F,1,314000.0,LONDON,LAMBETH,L,F,21/02/2020,SW12 0JP,54.0,92.304597,36.0,11.4,9.6,0.543902
4,8355F008-F016-55C5-E053-6B04A8C0D090,1,170000.0,BRAINTREE,BRAINTREE,L,F,01/02/2019,CM7 1TD,3.0,89.636307,13.0,9.7,8.3,0.452391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1219325,8355F009-2B4F-55C5-E053-6B04A8C0D090,47,79000.0,LEIGH,WIGAN,L,T,08/02/2019,WN7 1NG,12.0,72.452903,39.0,8.5,7.5,1.542007
1219326,8A78B2B0-784A-5CB0-E053-6B04A8C0F504,48,53000.0,BLACKWOOD,CAERPHILLY,F,S,12/04/2019,NP12 3LQ,1.0,0.000000,0.0,0.0,0.0,0.000000
1219327,8355F009-8A15-55C5-E053-6B04A8C0D090,55,70000.0,FLINT,FLINTSHIRE,F,T,11/02/2019,CH6 5DY,1.0,0.000000,0.0,0.0,0.0,0.000000
1219328,8A78B2B0-4326-5CB0-E053-6B04A8C0F504,81,95000.0,TODMORDEN,CALDERDALE,L,T,08/04/2019,OL14 8NL,1.0,88.443148,25.0,9.7,7.7,2.462274


In [26]:
data = data.dropna()
data

,unique_id,num_of_beds,price_paid,town,district,estate_type,property_type,deed_date,postcode,num_of_sec_sch,score_disadv,closest_disadv_prct,closest_aver_attain_eng,closest_aver_attain_math,closest_sch_distance
0,726BF13A-B9B6-0A46-E053-6C04A8C01D0D,1,100000.0,MANCHESTER,SALFORD,L,T,20/07/2018,M30 8EG,23.0,110.178111,96.0,0.5,0.9,0.671216
1,9B361206-34DE-1904-E053-6B04A8C0EEB5,1,79500.0,MANCHESTER,BURY,L,S,08/11/2019,M25 1ET,24.0,78.275669,15.0,10.8,9.4,0.320619
2,726BF13B-13CE-0A46-E053-6C04A8C01D0D,1,128000.0,NORWICH,NORWICH,L,F,20/04/2018,NR3 1SX,12.0,85.979811,20.0,11.5,9.8,0.584296
3,A71375FD-EBA1-7576-E053-6C04A8C0462F,1,314000.0,LONDON,LAMBETH,L,F,21/02/2020,SW12 0JP,54.0,92.304597,36.0,11.4,9.6,0.543902
4,8355F008-F016-55C5-E053-6B04A8C0D090,1,170000.0,BRAINTREE,BRAINTREE,L,F,01/02/2019,CM7 1TD,3.0,89.636307,13.0,9.7,8.3,0.452391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1219325,8355F009-2B4F-55C5-E053-6B04A8C0D090,47,79000.0,LEIGH,WIGAN,L,T,08/02/2019,WN7 1NG,12.0,72.452903,39.0,8.5,7.5,1.542007
1219326,8A78B2B0-784A-5CB0-E053-6B04A8C0F504,48,53000.0,BLACKWOOD,CAERPHILLY,F,S,12/04/2019,NP12 3LQ,1.0,0.000000,0.0,0.0,0.0,0.000000
1219327,8355F009-8A15-55C5-E053-6B04A8C0D090,55,70000.0,FLINT,FLINTSHIRE,F,T,11/02/2019,CH6 5DY,1.0,0.000000,0.0,0.0,0.0,0.000000
1219328,8A78B2B0-4326-5CB0-E053-6B04A8C0F504,81,95000.0,TODMORDEN,CALDERDALE,L,T,08/04/2019,OL14 8NL,1.0,88.443148,25.0,9.7,7.7,2.462274


In [27]:
#column to represent historic point: 01-2018=>1, 02-2018=>2 etc
#hist_mnth=12*(year-2018)+month
data['hist_mnth']=data['deed_date'].str.split('/').str[1].astype(int)+ 12*(data['deed_date'].str.split('/').str[2].astype(int)-2018)
data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,unique_id,num_of_beds,price_paid,town,district,estate_type,property_type,deed_date,postcode,num_of_sec_sch,score_disadv,closest_disadv_prct,closest_aver_attain_eng,closest_aver_attain_math,closest_sch_distance,hist_mnth
0,726BF13A-B9B6-0A46-E053-6C04A8C01D0D,1,100000.0,MANCHESTER,SALFORD,L,T,20/07/2018,M30 8EG,23.0,110.178111,96.0,0.5,0.9,0.671216,7
1,9B361206-34DE-1904-E053-6B04A8C0EEB5,1,79500.0,MANCHESTER,BURY,L,S,08/11/2019,M25 1ET,24.0,78.275669,15.0,10.8,9.4,0.320619,23
2,726BF13B-13CE-0A46-E053-6C04A8C01D0D,1,128000.0,NORWICH,NORWICH,L,F,20/04/2018,NR3 1SX,12.0,85.979811,20.0,11.5,9.8,0.584296,4
3,A71375FD-EBA1-7576-E053-6C04A8C0462F,1,314000.0,LONDON,LAMBETH,L,F,21/02/2020,SW12 0JP,54.0,92.304597,36.0,11.4,9.6,0.543902,26
4,8355F008-F016-55C5-E053-6B04A8C0D090,1,170000.0,BRAINTREE,BRAINTREE,L,F,01/02/2019,CM7 1TD,3.0,89.636307,13.0,9.7,8.3,0.452391,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1219325,8355F009-2B4F-55C5-E053-6B04A8C0D090,47,79000.0,LEIGH,WIGAN,L,T,08/02/2019,WN7 1NG,12.0,72.452903,39.0,8.5,7.5,1.542007,14
1219326,8A78B2B0-784A-5CB0-E053-6B04A8C0F504,48,53000.0,BLACKWOOD,CAERPHILLY,F,S,12/04/2019,NP12 3LQ,1.0,0.000000,0.0,0.0,0.0,0.000000,16
1219327,8355F009-8A15-55C5-E053-6B04A8C0D090,55,70000.0,FLINT,FLINTSHIRE,F,T,11/02/2019,CH6 5DY,1.0,0.000000,0.0,0.0,0.0,0.000000,14
1219328,8A78B2B0-4326-5CB0-E053-6B04A8C0F504,81,95000.0,TODMORDEN,CALDERDALE,L,T,08/04/2019,OL14 8NL,1.0,88.443148,25.0,9.7,7.7,2.462274,16


In [28]:
#remove the following outliers 
#remove sales transactions where number of beds is higher than 5
#remove property sales transactions where prices under 10,000 and over 1,000,000
data=data[(data['num_of_beds'] <=5) & (data['price_paid'] > 10000) & (data['price_paid'] <1000000) ]
data

,unique_id,num_of_beds,price_paid,town,district,estate_type,property_type,deed_date,postcode,num_of_sec_sch,score_disadv,closest_disadv_prct,closest_aver_attain_eng,closest_aver_attain_math,closest_sch_distance,hist_mnth
0,726BF13A-B9B6-0A46-E053-6C04A8C01D0D,1,100000.0,MANCHESTER,SALFORD,L,T,20/07/2018,M30 8EG,23.0,110.178111,96.0,0.5,0.9,0.671216,7
1,9B361206-34DE-1904-E053-6B04A8C0EEB5,1,79500.0,MANCHESTER,BURY,L,S,08/11/2019,M25 1ET,24.0,78.275669,15.0,10.8,9.4,0.320619,23
2,726BF13B-13CE-0A46-E053-6C04A8C01D0D,1,128000.0,NORWICH,NORWICH,L,F,20/04/2018,NR3 1SX,12.0,85.979811,20.0,11.5,9.8,0.584296,4
3,A71375FD-EBA1-7576-E053-6C04A8C0462F,1,314000.0,LONDON,LAMBETH,L,F,21/02/2020,SW12 0JP,54.0,92.304597,36.0,11.4,9.6,0.543902,26
4,8355F008-F016-55C5-E053-6B04A8C0D090,1,170000.0,BRAINTREE,BRAINTREE,L,F,01/02/2019,CM7 1TD,3.0,89.636307,13.0,9.7,8.3,0.452391,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1207297,87E1551E-0B78-6405-E053-6C04A8C0B2EE,5,570000.0,SHERBORNE,DORSET,F,D,08/04/2019,DT9 6NT,1.0,0.000000,0.0,0.0,0.0,0.000000,16
1207298,6DA08449-E135-30F2-E053-6B04A8C05F3B,5,890000.0,MANCHESTER,MANCHESTER,F,D,30/04/2018,M20 2WG,24.0,97.697793,97.0,1.9,2.8,1.639435,4
1207299,79A74E21-A664-1289-E053-6B04A8C01627,5,328000.0,HASTINGS,HASTINGS,F,S,24/09/2018,TN34 2EJ,6.0,193.121503,26.0,8.7,8.3,1.094252,9
1207300,666758D6-F397-3363-E053-6B04A8C0D74E,5,214000.0,PRESTON,SOUTH RIBBLE,F,D,02/02/2018,PR5 0AB,6.0,46.650826,27.0,9.5,8.1,2.156095,2


In [29]:
#extracting historic (3 months) median price per bedroom for each postal area
sqlstring='SELECT   median_hist_3mnth_after_removed_outliers.postcode_area_m_y,   median_hist_3mnth_after_removed_outliers.median_ppb_3mnths_ago FROM projectdata. median_hist_3mnth_after_removed_outliers'
hist_median_ppb_3mnth=db.sql(sqlstring)
df_hist_median_ppb_3mnth=pd.DataFrame(hist_median_ppb_3mnth)
df_hist_median_ppb_3mnth

,postcode_area_m_y,median_ppb_3mnths_ago
0,AL10_01_2018,118333.333333
1,AL10_01_2019,121666.666667
2,AL10_01_2020,121556.250000
3,AL10_02_2018,121666.666667
4,AL10_02_2019,126666.666667
...,...,...
61970,YO8_10_2019,60916.666667
61971,YO8_11_2018,62083.333333
61972,YO8_11_2019,58000.000000
61973,YO8_12_2018,65750.000000


In [30]:
#extracting historic (4 months) median price per bedroom for each postal area
sqlstring='SELECT  median_hist_4mnth_after_removed_outliers.postcode_area_m_y,  median_hist_4mnth_after_removed_outliers.median_ppb_4mnths_ago FROM projectdata.median_hist_4mnth_after_removed_outliers'
hist_median_ppb_4mnth=db.sql(sqlstring)
df_hist_median_ppb_4mnth=pd.DataFrame(hist_median_ppb_4mnth)
df_hist_median_ppb_4mnth

,postcode_area_m_y,median_ppb_4mnths_ago
0,AL10_01_2018,118333.333333
1,AL10_01_2019,114166.666667
2,AL10_01_2020,120000.000000
3,AL10_02_2018,121666.666667
4,AL10_02_2019,121666.666667
...,...,...
64030,YO8_10_2019,68750.000000
64031,YO8_11_2018,66000.000000
64032,YO8_11_2019,60916.666667
64033,YO8_12_2018,62083.333333


In [31]:
#fields for merging with historic median prices per bedroom 
data['postcode_area_m_y']=data['postcode'].str.split(' ').str[0]+'_'+data['deed_date'].str.split('/').str[1]+'_'+data['deed_date'].str.split('/').str[2]
data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,unique_id,num_of_beds,price_paid,town,district,estate_type,property_type,deed_date,postcode,num_of_sec_sch,score_disadv,closest_disadv_prct,closest_aver_attain_eng,closest_aver_attain_math,closest_sch_distance,hist_mnth,postcode_area_m_y
0,726BF13A-B9B6-0A46-E053-6C04A8C01D0D,1,100000.0,MANCHESTER,SALFORD,L,T,20/07/2018,M30 8EG,23.0,110.178111,96.0,0.5,0.9,0.671216,7,M30_07_2018
1,9B361206-34DE-1904-E053-6B04A8C0EEB5,1,79500.0,MANCHESTER,BURY,L,S,08/11/2019,M25 1ET,24.0,78.275669,15.0,10.8,9.4,0.320619,23,M25_11_2019
2,726BF13B-13CE-0A46-E053-6C04A8C01D0D,1,128000.0,NORWICH,NORWICH,L,F,20/04/2018,NR3 1SX,12.0,85.979811,20.0,11.5,9.8,0.584296,4,NR3_04_2018
3,A71375FD-EBA1-7576-E053-6C04A8C0462F,1,314000.0,LONDON,LAMBETH,L,F,21/02/2020,SW12 0JP,54.0,92.304597,36.0,11.4,9.6,0.543902,26,SW12_02_2020
4,8355F008-F016-55C5-E053-6B04A8C0D090,1,170000.0,BRAINTREE,BRAINTREE,L,F,01/02/2019,CM7 1TD,3.0,89.636307,13.0,9.7,8.3,0.452391,14,CM7_02_2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1207297,87E1551E-0B78-6405-E053-6C04A8C0B2EE,5,570000.0,SHERBORNE,DORSET,F,D,08/04/2019,DT9 6NT,1.0,0.000000,0.0,0.0,0.0,0.000000,16,DT9_04_2019
1207298,6DA08449-E135-30F2-E053-6B04A8C05F3B,5,890000.0,MANCHESTER,MANCHESTER,F,D,30/04/2018,M20 2WG,24.0,97.697793,97.0,1.9,2.8,1.639435,4,M20_04_2018
1207299,79A74E21-A664-1289-E053-6B04A8C01627,5,328000.0,HASTINGS,HASTINGS,F,S,24/09/2018,TN34 2EJ,6.0,193.121503,26.0,8.7,8.3,1.094252,9,TN34_09_2018
1207300,666758D6-F397-3363-E053-6B04A8C0D74E,5,214000.0,PRESTON,SOUTH RIBBLE,F,D,02/02/2018,PR5 0AB,6.0,46.650826,27.0,9.5,8.1,2.156095,2,PR5_02_2018


In [32]:
#merging the dataset with median historic prices per bedroom
data = pd.merge(data, df_hist_median_ppb_3mnth, how="left", on="postcode_area_m_y")
data = pd.merge(data, df_hist_median_ppb_4mnth, how="left", on="postcode_area_m_y")
data

,unique_id,num_of_beds,price_paid,town,district,estate_type,property_type,deed_date,postcode,num_of_sec_sch,score_disadv,closest_disadv_prct,closest_aver_attain_eng,closest_aver_attain_math,closest_sch_distance,hist_mnth,postcode_area_m_y,median_ppb_3mnths_ago,median_ppb_4mnths_ago
0,726BF13A-B9B6-0A46-E053-6C04A8C01D0D,1,100000.0,MANCHESTER,SALFORD,L,T,20/07/2018,M30 8EG,23.0,110.178111,96.0,0.5,0.9,0.671216,7,M30_07_2018,60000.000000,56187.500000
1,9B361206-34DE-1904-E053-6B04A8C0EEB5,1,79500.0,MANCHESTER,BURY,L,S,08/11/2019,M25 1ET,24.0,78.275669,15.0,10.8,9.4,0.320619,23,M25_11_2019,79166.666667,79000.000000
2,726BF13B-13CE-0A46-E053-6C04A8C01D0D,1,128000.0,NORWICH,NORWICH,L,F,20/04/2018,NR3 1SX,12.0,85.979811,20.0,11.5,9.8,0.584296,4,NR3_04_2018,73416.666667,64341.666667
3,A71375FD-EBA1-7576-E053-6C04A8C0462F,1,314000.0,LONDON,LAMBETH,L,F,21/02/2020,SW12 0JP,54.0,92.304597,36.0,11.4,9.6,0.543902,26,SW12_02_2020,285000.000000,326250.000000
4,8355F008-F016-55C5-E053-6B04A8C0D090,1,170000.0,BRAINTREE,BRAINTREE,L,F,01/02/2019,CM7 1TD,3.0,89.636307,13.0,9.7,8.3,0.452391,14,CM7_02_2019,101666.666667,101666.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1191132,87E1551E-0B78-6405-E053-6C04A8C0B2EE,5,570000.0,SHERBORNE,DORSET,F,D,08/04/2019,DT9 6NT,1.0,0.000000,0.0,0.0,0.0,0.000000,16,DT9_04_2019,87250.000000,92500.000000
1191133,6DA08449-E135-30F2-E053-6B04A8C05F3B,5,890000.0,MANCHESTER,MANCHESTER,F,D,30/04/2018,M20 2WG,24.0,97.697793,97.0,1.9,2.8,1.639435,4,M20_04_2018,105000.000000,107500.000000
1191134,79A74E21-A664-1289-E053-6B04A8C01627,5,328000.0,HASTINGS,HASTINGS,F,S,24/09/2018,TN34 2EJ,6.0,193.121503,26.0,8.7,8.3,1.094252,9,TN34_09_2018,89750.000000,92500.000000
1191135,666758D6-F397-3363-E053-6B04A8C0D74E,5,214000.0,PRESTON,SOUTH RIBBLE,F,D,02/02/2018,PR5 0AB,6.0,46.650826,27.0,9.5,8.1,2.156095,2,PR5_02_2018,57750.000000,57750.000000


In [33]:
#checking where median_ppb_3mnths_ago is NAN 
data_nan_median_historic_ppb = data[(data['median_ppb_3mnths_ago'].isnull())]
data_nan_median_historic_ppb

,unique_id,num_of_beds,price_paid,town,district,estate_type,property_type,deed_date,postcode,num_of_sec_sch,score_disadv,closest_disadv_prct,closest_aver_attain_eng,closest_aver_attain_math,closest_sch_distance,hist_mnth,postcode_area_m_y,median_ppb_3mnths_ago,median_ppb_4mnths_ago
544,75050A84-D2F3-9A88-E053-6B04A8C02390,1,97000.0,LISKEARD,CORNWALL,F,S,20/07/2018,PL14 3QN,1.0,199.732649,39.0,8.8,7.6,0.878650,7,PL14_07_2018,NaN,NaN
578,773788C2-5C25-2CE4-E053-6C04A8C05E57,1,134500.0,BOSCASTLE,CORNWALL,F,T,16/08/2018,PL35 0AL,1.0,0.000000,0.0,0.0,0.0,0.000000,8,PL35_08_2018,NaN,NaN
850,8F1B26BD-E807-53DB-E053-6C04A8C03649,1,97000.0,MANCHESTER,MANCHESTER,L,F,24/05/2019,M13 0DB,27.0,89.273531,70.0,8.3,8.1,1.333832,17,M13_05_2019,NaN,55000.000000
1238,7E86B6FB-1C9D-458C-E053-6B04A8C0C84C,1,450000.0,LONDON,ISLINGTON,L,F,17/08/2018,WC1X 9AA,80.0,106.458651,64.0,11.2,9.0,0.669889,8,WC1X_08_2018,NaN,NaN
1259,7C2D0700-652F-4963-E053-6B04A8C07B97,1,129000.0,CHATHILL,NORTHUMBERLAND,F,T,15/10/2018,NE67 5BP,1.0,0.000000,0.0,0.0,0.0,0.000000,10,NE67_10_2018,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1190695,75050A85-4C95-9A88-E053-6B04A8C02390,5,625000.0,PETERBOROUGH,SOUTH KESTEVEN,F,D,18/07/2018,PE6 8NJ,2.0,60.150836,18.0,9.1,8.0,0.480194,7,PE6_07_2018,NaN,NaN
1190729,726BF13A-7B6D-0A46-E053-6C04A8C01D0D,5,304600.0,LISKEARD,CORNWALL,F,D,20/06/2018,PL14 6DP,1.0,197.019215,39.0,8.8,7.6,0.948225,6,PL14_06_2018,NaN,60125.000000
1190803,8A78B2AF-D008-5CB0-E053-6B04A8C0F504,5,500000.0,HALTWHISTLE,NORTHUMBERLAND,F,D,17/04/2019,NE49 9NW,1.0,0.000000,0.0,0.0,0.0,0.000000,16,NE49_04_2019,NaN,73750.000000
1190959,8CAC1318-7C4B-0253-E053-6B04A8C08E51,5,525000.0,RIDING MILL,NORTHUMBERLAND,F,T,19/06/2019,NE44 6HR,1.0,0.000000,0.0,0.0,0.0,0.000000,18,NE44_06_2019,NaN,NaN


In [34]:
#checking where median_ppb_4mnths_ago is NAN 
data_nan_median_historic_ppb = data[(data['median_ppb_4mnths_ago'].isnull())]
data_nan_median_historic_ppb

,unique_id,num_of_beds,price_paid,town,district,estate_type,property_type,deed_date,postcode,num_of_sec_sch,score_disadv,closest_disadv_prct,closest_aver_attain_eng,closest_aver_attain_math,closest_sch_distance,hist_mnth,postcode_area_m_y,median_ppb_3mnths_ago,median_ppb_4mnths_ago
73,79A74E22-03DF-1289-E053-6B04A8C01627,1,128000.0,PORTSMOUTH,PORTSMOUTH,L,S,31/08/2018,PO2 9RD,17.0,130.174837,30.0,9.0,9.7,0.197140,8,PO2_08_2018,75000.000000,NaN
459,919FEC05-7458-9A90-E053-6C04A8C0A300,1,172500.0,MANCHESTER,MANCHESTER,L,F,25/07/2019,M4 1LY,30.0,66.237801,38.0,9.4,8.5,2.195914,19,M4_07_2019,270000.000000,NaN
544,75050A84-D2F3-9A88-E053-6B04A8C02390,1,97000.0,LISKEARD,CORNWALL,F,S,20/07/2018,PL14 3QN,1.0,199.732649,39.0,8.8,7.6,0.878650,7,PL14_07_2018,NaN,NaN
578,773788C2-5C25-2CE4-E053-6C04A8C05E57,1,134500.0,BOSCASTLE,CORNWALL,F,T,16/08/2018,PL35 0AL,1.0,0.000000,0.0,0.0,0.0,0.000000,8,PL35_08_2018,NaN,NaN
780,773788C2-AD4C-2CE4-E053-6C04A8C05E57,1,189950.0,FAREHAM,WINCHESTER,F,T,17/08/2018,PO15 7HQ,6.0,61.857206,90.0,4.4,4.0,2.675842,8,PO15_08_2018,101666.666667,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1190090,75050A85-22AB-9A88-E053-6B04A8C02390,5,252000.0,PORTSMOUTH,PORTSMOUTH,F,T,09/07/2018,PO2 9EJ,16.0,141.989411,31.0,8.8,7.9,0.627369,7,PO2_07_2018,NaN,NaN
1190695,75050A85-4C95-9A88-E053-6B04A8C02390,5,625000.0,PETERBOROUGH,SOUTH KESTEVEN,F,D,18/07/2018,PE6 8NJ,2.0,60.150836,18.0,9.1,8.0,0.480194,7,PE6_07_2018,NaN,NaN
1190724,75050A85-A399-9A88-E053-6B04A8C02390,5,458000.0,BANWELL,NORTH SOMERSET,F,S,05/07/2018,BS29 6AG,4.0,28.638033,19.0,10.6,10.1,4.463332,7,BS29_07_2018,72000.000000,NaN
1190959,8CAC1318-7C4B-0253-E053-6B04A8C08E51,5,525000.0,RIDING MILL,NORTHUMBERLAND,F,T,19/06/2019,NE44 6HR,1.0,0.000000,0.0,0.0,0.0,0.000000,18,NE44_06_2019,NaN,NaN


In [35]:
#removing from the dataset those transactions which do not have the historic median ppb info 
data = data.dropna()
data

,unique_id,num_of_beds,price_paid,town,district,estate_type,property_type,deed_date,postcode,num_of_sec_sch,score_disadv,closest_disadv_prct,closest_aver_attain_eng,closest_aver_attain_math,closest_sch_distance,hist_mnth,postcode_area_m_y,median_ppb_3mnths_ago,median_ppb_4mnths_ago
0,726BF13A-B9B6-0A46-E053-6C04A8C01D0D,1,100000.0,MANCHESTER,SALFORD,L,T,20/07/2018,M30 8EG,23.0,110.178111,96.0,0.5,0.9,0.671216,7,M30_07_2018,60000.000000,56187.500000
1,9B361206-34DE-1904-E053-6B04A8C0EEB5,1,79500.0,MANCHESTER,BURY,L,S,08/11/2019,M25 1ET,24.0,78.275669,15.0,10.8,9.4,0.320619,23,M25_11_2019,79166.666667,79000.000000
2,726BF13B-13CE-0A46-E053-6C04A8C01D0D,1,128000.0,NORWICH,NORWICH,L,F,20/04/2018,NR3 1SX,12.0,85.979811,20.0,11.5,9.8,0.584296,4,NR3_04_2018,73416.666667,64341.666667
3,A71375FD-EBA1-7576-E053-6C04A8C0462F,1,314000.0,LONDON,LAMBETH,L,F,21/02/2020,SW12 0JP,54.0,92.304597,36.0,11.4,9.6,0.543902,26,SW12_02_2020,285000.000000,326250.000000
4,8355F008-F016-55C5-E053-6B04A8C0D090,1,170000.0,BRAINTREE,BRAINTREE,L,F,01/02/2019,CM7 1TD,3.0,89.636307,13.0,9.7,8.3,0.452391,14,CM7_02_2019,101666.666667,101666.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1191132,87E1551E-0B78-6405-E053-6C04A8C0B2EE,5,570000.0,SHERBORNE,DORSET,F,D,08/04/2019,DT9 6NT,1.0,0.000000,0.0,0.0,0.0,0.000000,16,DT9_04_2019,87250.000000,92500.000000
1191133,6DA08449-E135-30F2-E053-6B04A8C05F3B,5,890000.0,MANCHESTER,MANCHESTER,F,D,30/04/2018,M20 2WG,24.0,97.697793,97.0,1.9,2.8,1.639435,4,M20_04_2018,105000.000000,107500.000000
1191134,79A74E21-A664-1289-E053-6B04A8C01627,5,328000.0,HASTINGS,HASTINGS,F,S,24/09/2018,TN34 2EJ,6.0,193.121503,26.0,8.7,8.3,1.094252,9,TN34_09_2018,89750.000000,92500.000000
1191135,666758D6-F397-3363-E053-6B04A8C0D74E,5,214000.0,PRESTON,SOUTH RIBBLE,F,D,02/02/2018,PR5 0AB,6.0,46.650826,27.0,9.5,8.1,2.156095,2,PR5_02_2018,57750.000000,57750.000000


In [36]:
#calculate each price per bedroom for comparing with historic median ppd
data['this_price_per_bedroom']=data['price_paid']/data['num_of_beds']
data['compare_ppd_re_hist_median_3mnths']=100*(data['this_price_per_bedroom']-data['median_ppb_3mnths_ago'])/data['median_ppb_3mnths_ago']
data['compare_ppd_re_hist_median_4mnths']=100*(data['this_price_per_bedroom']-data['median_ppb_4mnths_ago'])/data['median_ppb_4mnths_ago']
data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

,unique_id,num_of_beds,price_paid,town,district,estate_type,property_type,deed_date,postcode,num_of_sec_sch,score_disadv,closest_disadv_prct,closest_aver_attain_eng,closest_aver_attain_math,closest_sch_distance,hist_mnth,postcode_area_m_y,median_ppb_3mnths_ago,median_ppb_4mnths_ago,this_price_per_bedroom,compare_ppd_re_hist_median_3mnths,compare_ppd_re_hist_median_4mnths
0,726BF13A-B9B6-0A46-E053-6C04A8C01D0D,1,100000.0,MANCHESTER,SALFORD,L,T,20/07/2018,M30 8EG,23.0,110.178111,96.0,0.5,0.9,0.671216,7,M30_07_2018,60000.000000,56187.500000,100000.0,66.666667,77.975528
1,9B361206-34DE-1904-E053-6B04A8C0EEB5,1,79500.0,MANCHESTER,BURY,L,S,08/11/2019,M25 1ET,24.0,78.275669,15.0,10.8,9.4,0.320619,23,M25_11_2019,79166.666667,79000.000000,79500.0,0.421053,0.632911
2,726BF13B-13CE-0A46-E053-6C04A8C01D0D,1,128000.0,NORWICH,NORWICH,L,F,20/04/2018,NR3 1SX,12.0,85.979811,20.0,11.5,9.8,0.584296,4,NR3_04_2018,73416.666667,64341.666667,128000.0,74.347333,98.937961
3,A71375FD-EBA1-7576-E053-6C04A8C0462F,1,314000.0,LONDON,LAMBETH,L,F,21/02/2020,SW12 0JP,54.0,92.304597,36.0,11.4,9.6,0.543902,26,SW12_02_2020,285000.000000,326250.000000,314000.0,10.175439,-3.754789
4,8355F008-F016-55C5-E053-6B04A8C0D090,1,170000.0,BRAINTREE,BRAINTREE,L,F,01/02/2019,CM7 1TD,3.0,89.636307,13.0,9.7,8.3,0.452391,14,CM7_02_2019,101666.666667,101666.666667,170000.0,67.213115,67.213115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1191132,87E1551E-0B78-6405-E053-6C04A8C0B2EE,5,570000.0,SHERBORNE,DORSET,F,D,08/04/2019,DT9 6NT,1.0,0.000000,0.0,0.0,0.0,0.000000,16,DT9_04_2019,87250.000000,92500.000000,114000.0,30.659026,23.243243
1191133,6DA08449-E135-30F2-E053-6B04A8C05F3B,5,890000.0,MANCHESTER,MANCHESTER,F,D,30/04/2018,M20 2WG,24.0,97.697793,97.0,1.9,2.8,1.639435,4,M20_04_2018,105000.000000,107500.000000,178000.0,69.523810,65.581395
1191134,79A74E21-A664-1289-E053-6B04A8C01627,5,328000.0,HASTINGS,HASTINGS,F,S,24/09/2018,TN34 2EJ,6.0,193.121503,26.0,8.7,8.3,1.094252,9,TN34_09_2018,89750.000000,92500.000000,65600.0,-26.908078,-29.081081
1191135,666758D6-F397-3363-E053-6B04A8C0D74E,5,214000.0,PRESTON,SOUTH RIBBLE,F,D,02/02/2018,PR5 0AB,6.0,46.650826,27.0,9.5,8.1,2.156095,2,PR5_02_2018,57750.000000,57750.000000,42800.0,-25.887446,-25.887446


In [37]:
#extracting data on distances from each sales postcode to three locations (in London, Birmingham, Manchester) 
sqlstring='SELECT  distances_to_three_locations.sales_postcode,  distances_to_three_locations.distances_Lond,  distances_to_three_locations.distances_Manch,  distances_to_three_locations.distances_Birming FROM projectdata.distances_to_three_locations'
data_on_distances=db.sql(sqlstring)
df_data_on_distances=pd.DataFrame(data_on_distances)
df_data_on_distances.rename(columns = {'sales_postcode':'postcode'}, inplace = True)
df_data_on_distances=df_data_on_distances.round({'distances_Lond': 5, 'distances_Manch': 5,'distances_Birming': 5 })
#--------------------------------------------------
df_data_on_distances['add_num']=df_data_on_distances['postcode'].str.split(' ').str[1].str[-1:]
df_data_on_distances['add_num']=df_data_on_distances['add_num'].map({'A':'1','B':'2','C':'3','D':'4','E':'5','F':'6','G':'7','H':'8','I':'9','J':'10','K':'11','L':'12','M':'13','N':'14','O':'15','P':'16','Q':'17','R':'18','S':'19','T':'20','U':'21','V':'22','W':'23','X':'24','Y':'25','Z':'26'})
df_data_on_distances['add_num']=df_data_on_distances['add_num'].astype(float)
df_data_on_distances['distances_Lond']=df_data_on_distances['distances_Lond']+0.0001*df_data_on_distances['add_num']
df_data_on_distances['distances_Lond']=(df_data_on_distances['distances_Lond']*1000000).astype(int)*100
df_data_on_distances['distances_Manch']=(df_data_on_distances['distances_Manch']*1000000).astype(int)*100
df_data_on_distances['distances_Birming']=(df_data_on_distances['distances_Birming']*1000000).astype(int)*100
df_data_on_distances.drop(['add_num'],  axis='columns', inplace=True)
df_data_on_distances


,postcode,distances_Lond,distances_Manch,distances_Birming
0,AL1 1AJ,2799619000,23266136000,13526048000
1,AL1 1AR,2721051000,23352395000,13618984000
2,AL1 1AT,2749614000,23323872000,13592915000
3,AL1 1AU,2743726900,23329416000,13597375000
4,AL1 1DG,2875380000,23179172000,13422322000
...,...,...,...,...
580927,YO8 9XL,25831381000,8403574000,15374529000
580928,YO8 9XN,25824568900,8402270000,15368758000
580929,YO8 9XP,25846332000,8420687000,15395483000
580930,YO8 9YB,25968414000,7884438000,15173418000


In [38]:
#checking if the combination of the three distances uniquely defines each postcode
df_data_on_distances['location_key']=((df_data_on_distances['distances_Lond']*0.01).astype(int)).astype(str) + '_' + ((df_data_on_distances['distances_Manch']*0.01).astype(int)).astype(str)+ '_' + ((df_data_on_distances['distances_Birming']*0.01).astype(int)).astype(str)
g_dist_test = df_data_on_distances.groupby(['location_key'])['postcode'].count().reset_index()
g_dist_test_if_not_unique=g_dist_test[(g_dist_test['postcode']>1)]
g_dist_test_if_not_unique

,location_key,postcode


In [39]:
data = pd.merge(data, df_data_on_distances, how="left", on="postcode")
data

,unique_id,num_of_beds,price_paid,town,district,estate_type,property_type,deed_date,postcode,num_of_sec_sch,score_disadv,closest_disadv_prct,closest_aver_attain_eng,closest_aver_attain_math,closest_sch_distance,hist_mnth,postcode_area_m_y,median_ppb_3mnths_ago,median_ppb_4mnths_ago,this_price_per_bedroom,compare_ppd_re_hist_median_3mnths,compare_ppd_re_hist_median_4mnths,distances_Lond,distances_Manch,distances_Birming,location_key
0,726BF13A-B9B6-0A46-E053-6C04A8C01D0D,1,100000.0,MANCHESTER,SALFORD,L,T,20/07/2018,M30 8EG,23.0,110.178111,96.0,0.5,0.9,0.671216,7,M30_07_2018,60000.000000,56187.500000,100000.0,66.666667,77.975528,26510768000,718540000,11622296000,265107680_7185400_116222960
1,9B361206-34DE-1904-E053-6B04A8C0EEB5,1,79500.0,MANCHESTER,BURY,L,S,08/11/2019,M25 1ET,24.0,78.275669,15.0,10.8,9.4,0.320619,23,M25_11_2019,79166.666667,79000.000000,79500.0,0.421053,0.632911,26539970000,542647000,11902983000,265399700_5426470_119029830
2,726BF13B-13CE-0A46-E053-6C04A8C01D0D,1,128000.0,NORWICH,NORWICH,L,F,20/04/2018,NR3 1SX,12.0,85.979811,20.0,11.5,9.8,0.584296,4,NR3_04_2018,73416.666667,64341.666667,128000.0,74.347333,98.937961,16016690000,25505656000,21724217000,160166900_255056560_217242170
3,A71375FD-EBA1-7576-E053-6C04A8C0462F,1,314000.0,LONDON,LAMBETH,L,F,21/02/2020,SW12 0JP,54.0,92.304597,36.0,11.4,9.6,0.543902,26,SW12_02_2020,285000.000000,326250.000000,314000.0,10.175439,-3.754789,770794900,26771245000,16701868000,7707949_267712450_167018680
4,8355F008-F016-55C5-E053-6B04A8C0D090,1,170000.0,BRAINTREE,BRAINTREE,L,F,01/02/2019,CM7 1TD,3.0,89.636307,13.0,9.7,8.3,0.452391,14,CM7_02_2019,101666.666667,101666.666667,170000.0,67.213115,67.213115,6296533000,26014878000,18039516000,62965330_260148780_180395160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1182761,87E1551E-0B78-6405-E053-6C04A8C0B2EE,5,570000.0,SHERBORNE,DORSET,F,D,08/04/2019,DT9 6NT,1.0,0.000000,0.0,0.0,0.0,0.000000,16,DT9_04_2019,87250.000000,92500.000000,114000.0,30.659026,23.243243,18126308000,28825475000,18214946000,181263080_288254750_182149460
1182762,6DA08449-E135-30F2-E053-6B04A8C05F3B,5,890000.0,MANCHESTER,MANCHESTER,F,D,30/04/2018,M20 2WG,24.0,97.697793,97.0,1.9,2.8,1.639435,4,M20_04_2018,105000.000000,107500.000000,178000.0,69.523810,65.581395,25424316000,711278000,10659618000,254243160_7112780_106596180
1182763,79A74E21-A664-1289-E053-6B04A8C01627,5,328000.0,HASTINGS,HASTINGS,F,S,24/09/2018,TN34 2EJ,6.0,193.121503,26.0,8.7,8.3,1.094252,9,TN34_09_2018,89750.000000,92500.000000,65600.0,-26.908078,-29.081081,8815989000,34831047000,24742056000,88159890_348310470_247420560
1182764,666758D6-F397-3363-E053-6B04A8C0D74E,5,214000.0,PRESTON,SOUTH RIBBLE,F,D,02/02/2018,PR5 0AB,6.0,46.650826,27.0,9.5,8.1,2.156095,2,PR5_02_2018,57750.000000,57750.000000,42800.0,-25.887446,-25.887446,29718009000,3717254000,14712579000,297180090_37172540_147125790


In [40]:
#extracting postcodes lat and long from HarperDB (tables 'postcodes' and 'other postcodes')
#some stored in the DB postcodes are wrongly formatted (without a space) - reformatting those
sqlstring='SELECT postcodes.pcd, postcodes.lat, postcodes.long FROM projectdata.postcodes'
postcodes=db.sql(sqlstring)
df_postcodes=pd.DataFrame(postcodes)
df_postcodes['test_pcd']=df_postcodes['pcd'].str.split(' ').str[0]+ " " + df_postcodes['pcd'].str.split(' ').str[1]
data_pc1 = df_postcodes[(df_postcodes['test_pcd'].isnull())]
data_pc1["test_pcd"]= data_pc1['pcd'].str[:4]+" "+data_pc1['pcd'].str[-3:]
data_pc2 = df_postcodes.dropna(subset=['test_pcd'])
sqlstring='SELECT other_postcodes.postcode, other_postcodes.lat, other_postcodes.long FROM projectdata.other_postcodes'
pc3=db.sql(sqlstring)
data_pc3=pd.DataFrame(pc3)
frames = [data_pc1, data_pc2,]
df_postcodes=pd.concat(frames)
df_postcodes.rename(columns = {'test_pcd':'postcode'}, inplace = True)
df_postcodes.drop(['pcd' ],  axis='columns', inplace=True)
frames = [df_postcodes, data_pc3]
df_postcodes=pd.concat(frames)
df_postcodes

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,lat,long,postcode
2655,57.148232,-2.096648,AB10 1AA
2656,57.149606,-2.096916,AB10 1AB
2657,57.148125,-2.095540,AB10 1AD
2658,57.148707,-2.097806,AB10 1AF
2659,57.149051,-2.097004,AB10 1AG
...,...,...,...
10834,51.525860,-0.200157,W9 3PL
10835,51.527378,-0.198281,W9 3PQ
10836,51.525373,-0.203564,W9 3QF
10837,51.527051,-0.198077,W9 3SP


In [41]:
data = pd.merge(data, df_postcodes, how="left", on="postcode")
data

,unique_id,num_of_beds,price_paid,town,district,estate_type,property_type,deed_date,postcode,num_of_sec_sch,score_disadv,closest_disadv_prct,closest_aver_attain_eng,closest_aver_attain_math,closest_sch_distance,hist_mnth,postcode_area_m_y,median_ppb_3mnths_ago,median_ppb_4mnths_ago,this_price_per_bedroom,compare_ppd_re_hist_median_3mnths,compare_ppd_re_hist_median_4mnths,distances_Lond,distances_Manch,distances_Birming,location_key,lat,long
0,726BF13A-B9B6-0A46-E053-6C04A8C01D0D,1,100000.0,MANCHESTER,SALFORD,L,T,20/07/2018,M30 8EG,23.0,110.178111,96.0,0.5,0.9,0.671216,7,M30_07_2018,60000.000000,56187.500000,100000.0,66.666667,77.975528,26510768000,718540000,11622296000,265107680_7185400_116222960,53.489825,-2.358860
1,9B361206-34DE-1904-E053-6B04A8C0EEB5,1,79500.0,MANCHESTER,BURY,L,S,08/11/2019,M25 1ET,24.0,78.275669,15.0,10.8,9.4,0.320619,23,M25_11_2019,79166.666667,79000.000000,79500.0,0.421053,0.632911,26539970000,542647000,11902983000,265399700_5426470_119029830,53.528583,-2.270335
2,726BF13B-13CE-0A46-E053-6C04A8C01D0D,1,128000.0,NORWICH,NORWICH,L,F,20/04/2018,NR3 1SX,12.0,85.979811,20.0,11.5,9.8,0.584296,4,NR3_04_2018,73416.666667,64341.666667,128000.0,74.347333,98.937961,16016690000,25505656000,21724217000,160166900_255056560_217242170,52.637255,1.298314
3,A71375FD-EBA1-7576-E053-6C04A8C0462F,1,314000.0,LONDON,LAMBETH,L,F,21/02/2020,SW12 0JP,54.0,92.304597,36.0,11.4,9.6,0.543902,26,SW12_02_2020,285000.000000,326250.000000,314000.0,10.175439,-3.754789,770794900,26771245000,16701868000,7707949_267712450_167018680,51.446749,-0.137157
4,8355F008-F016-55C5-E053-6B04A8C0D090,1,170000.0,BRAINTREE,BRAINTREE,L,F,01/02/2019,CM7 1TD,3.0,89.636307,13.0,9.7,8.3,0.452391,14,CM7_02_2019,101666.666667,101666.666667,170000.0,67.213115,67.213115,6296533000,26014878000,18039516000,62965330_260148780_180395160,51.868112,0.544362
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1182761,87E1551E-0B78-6405-E053-6C04A8C0B2EE,5,570000.0,SHERBORNE,DORSET,F,D,08/04/2019,DT9 6NT,1.0,0.000000,0.0,0.0,0.0,0.000000,16,DT9_04_2019,87250.000000,92500.000000,114000.0,30.659026,23.243243,18126308000,28825475000,18214946000,181263080_288254750_182149460,50.896436,-2.576097
1182762,6DA08449-E135-30F2-E053-6B04A8C05F3B,5,890000.0,MANCHESTER,MANCHESTER,F,D,30/04/2018,M20 2WG,24.0,97.697793,97.0,1.9,2.8,1.639435,4,M20_04_2018,105000.000000,107500.000000,178000.0,69.523810,65.581395,25424316000,711278000,10659618000,254243160_7112780_106596180,53.417210,-2.246727
1182763,79A74E21-A664-1289-E053-6B04A8C01627,5,328000.0,HASTINGS,HASTINGS,F,S,24/09/2018,TN34 2EJ,6.0,193.121503,26.0,8.7,8.3,1.094252,9,TN34_09_2018,89750.000000,92500.000000,65600.0,-26.908078,-29.081081,8815989000,34831047000,24742056000,88159890_348310470_247420560,50.868112,0.567342
1182764,666758D6-F397-3363-E053-6B04A8C0D74E,5,214000.0,PRESTON,SOUTH RIBBLE,F,D,02/02/2018,PR5 0AB,6.0,46.650826,27.0,9.5,8.1,2.156095,2,PR5_02_2018,57750.000000,57750.000000,42800.0,-25.887446,-25.887446,29718009000,3717254000,14712579000,297180090_37172540_147125790,53.731672,-2.624259


In [ ]:
#removing data which has current 'price-per-bedroom' over 100% comparing to the historic medial 'price-per-bedroom'
#data = data[(data['compare_ppd_re_hist_median_3mnths'] <=100) & (data['compare_ppd_re_hist_median_3mnths'] >=-100) ]
#data = data[(data['compare_ppd_re_hist_median_4mnths'] <=100) & (data['compare_ppd_re_hist_median_4mnths'] >=-100) ]
#data

,unique_id,num_of_beds,price_paid,town,district,estate_type,property_type,deed_date,postcode,num_of_sec_sch,score_disadv,closest_disadv_prct,closest_aver_attain_eng,closest_aver_attain_math,closest_sch_distance,hist_mnth,postcode_area_m_y,median_ppb_3mnths_ago,median_ppb_4mnths_ago,this_price_per_bedroom,compare_ppd_re_hist_median_3mnths,compare_ppd_re_hist_median_4mnths,distances_Lond,distances_Manch,distances_Birming,location_key,lat,long
0,726BF13A-B9B6-0A46-E053-6C04A8C01D0D,1,100000.0,MANCHESTER,SALFORD,L,T,20/07/2018,M30 8EG,23.0,110.178111,96.0,0.5,0.9,0.671216,7,M30_07_2018,60250.00000,56187.50000,100000.000000,65.975104,77.975528,26510768000,718540000,11622296000,265107680_7185400_116222960,53.489825,-2.358860
1,9B361206-34DE-1904-E053-6B04A8C0EEB5,1,79500.0,MANCHESTER,BURY,L,S,08/11/2019,M25 1ET,24.0,78.275669,15.0,10.8,9.4,0.320619,23,M25_11_2019,79166.66667,79000.00000,79500.000000,0.421053,0.632911,26539970000,542647000,11902983000,265399700_5426470_119029830,53.528583,-2.270335
2,726BF13B-13CE-0A46-E053-6C04A8C01D0D,1,128000.0,NORWICH,NORWICH,L,F,20/04/2018,NR3 1SX,12.0,85.979811,20.0,11.5,9.8,0.584296,4,NR3_04_2018,73416.66667,64341.66667,128000.000000,74.347333,98.937961,16016690000,25505656000,21724217000,160166900_255056560_217242170,52.637255,1.298314
3,A71375FD-EBA1-7576-E053-6C04A8C0462F,1,314000.0,LONDON,LAMBETH,L,F,21/02/2020,SW12 0JP,54.0,92.304597,36.0,11.4,9.6,0.543902,26,SW12_02_2020,295000.00000,330000.00000,314000.000000,6.440678,-4.848485,770794900,26771245000,16701868000,7707949_267712450_167018680,51.446749,-0.137157
4,8355F008-F016-55C5-E053-6B04A8C0D090,1,170000.0,BRAINTREE,BRAINTREE,L,F,01/02/2019,CM7 1TD,3.0,89.636307,13.0,9.7,8.3,0.452391,14,CM7_02_2019,101666.66670,101666.66670,170000.000000,67.213115,67.213115,6296533000,26014878000,18039516000,62965330_260148780_180395160,51.868112,0.544362
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1210775,8355F009-2B4F-55C5-E053-6B04A8C0D090,47,79000.0,LEIGH,WIGAN,L,T,08/02/2019,WN7 1NG,12.0,72.452903,39.0,8.5,7.5,1.542007,14,WN7_02_2019,37625.00000,40499.37500,1680.851064,-95.532622,-95.849686,27236922000,1773271000,12075555000,272369220_17732710_120755550,53.502594,-2.516861
1210776,8A78B2B0-784A-5CB0-E053-6B04A8C0F504,48,53000.0,BLACKWOOD,CAERPHILLY,F,S,12/04/2019,NP12 3LQ,1.0,0.000000,0.0,0.0,0.0,0.000000,16,NP12_04_2019,40000.00000,50500.00000,1104.166667,-97.239583,-97.813531,21104985000,21110074000,12654123000,211049850_211100740_126541230,51.674615,-3.212091
1210777,8355F009-8A15-55C5-E053-6B04A8C0D090,55,70000.0,FLINT,FLINTSHIRE,F,T,11/02/2019,CH6 5DY,1.0,0.000000,0.0,0.0,0.0,0.000000,14,CH6_02_2019,41333.33333,45416.66667,1272.727273,-96.920821,-97.197665,27804846000,6355589000,11793603000,278048460_63555890_117936030,53.245090,-3.123599
1210778,8A78B2B0-4326-5CB0-E053-6B04A8C0F504,81,95000.0,TODMORDEN,CALDERDALE,L,T,08/04/2019,OL14 8NL,1.0,88.443148,25.0,9.7,7.7,2.462274,16,OL14_04_2019,54000.00000,56650.00000,1172.839506,-97.828075,-97.929674,28051597900,2906478000,14022715000,280515979_29064780_140227150,53.734079,-2.140287


In [ ]:
#remove columns: 'this_price_per_bedroom',	'compare_ppd_re_hist_median_3mnths',	'compare_ppd_re_hist_median_4mnths' (these were calculated using the known sales prices)
#data.drop(['this_price_per_bedroom' ,'compare_ppd_re_hist_median_3mnths', 'compare_ppd_re_hist_median_4mnths'],  axis='columns', inplace=True)
#data

,unique_id,num_of_beds,price_paid,town,district,estate_type,property_type,deed_date,postcode,num_of_sec_sch,score_disadv,closest_disadv_prct,closest_aver_attain_eng,closest_aver_attain_math,closest_sch_distance,hist_mnth,postcode_area_m_y,median_ppb_3mnths_ago,median_ppb_4mnths_ago,distances_Lond,distances_Manch,distances_Birming,location_key,lat,long
0,726BF13A-B9B6-0A46-E053-6C04A8C01D0D,1,100000.0,MANCHESTER,SALFORD,L,T,20/07/2018,M30 8EG,23.0,110.178111,96.0,0.5,0.9,0.671216,7,M30_07_2018,60250.00000,56187.50000,26510768000,718540000,11622296000,265107680_7185400_116222960,53.489825,-2.358860
1,9B361206-34DE-1904-E053-6B04A8C0EEB5,1,79500.0,MANCHESTER,BURY,L,S,08/11/2019,M25 1ET,24.0,78.275669,15.0,10.8,9.4,0.320619,23,M25_11_2019,79166.66667,79000.00000,26539970000,542647000,11902983000,265399700_5426470_119029830,53.528583,-2.270335
2,726BF13B-13CE-0A46-E053-6C04A8C01D0D,1,128000.0,NORWICH,NORWICH,L,F,20/04/2018,NR3 1SX,12.0,85.979811,20.0,11.5,9.8,0.584296,4,NR3_04_2018,73416.66667,64341.66667,16016690000,25505656000,21724217000,160166900_255056560_217242170,52.637255,1.298314
3,A71375FD-EBA1-7576-E053-6C04A8C0462F,1,314000.0,LONDON,LAMBETH,L,F,21/02/2020,SW12 0JP,54.0,92.304597,36.0,11.4,9.6,0.543902,26,SW12_02_2020,295000.00000,330000.00000,770794900,26771245000,16701868000,7707949_267712450_167018680,51.446749,-0.137157
4,8355F008-F016-55C5-E053-6B04A8C0D090,1,170000.0,BRAINTREE,BRAINTREE,L,F,01/02/2019,CM7 1TD,3.0,89.636307,13.0,9.7,8.3,0.452391,14,CM7_02_2019,101666.66670,101666.66670,6296533000,26014878000,18039516000,62965330_260148780_180395160,51.868112,0.544362
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1210775,8355F009-2B4F-55C5-E053-6B04A8C0D090,47,79000.0,LEIGH,WIGAN,L,T,08/02/2019,WN7 1NG,12.0,72.452903,39.0,8.5,7.5,1.542007,14,WN7_02_2019,37625.00000,40499.37500,27236922000,1773271000,12075555000,272369220_17732710_120755550,53.502594,-2.516861
1210776,8A78B2B0-784A-5CB0-E053-6B04A8C0F504,48,53000.0,BLACKWOOD,CAERPHILLY,F,S,12/04/2019,NP12 3LQ,1.0,0.000000,0.0,0.0,0.0,0.000000,16,NP12_04_2019,40000.00000,50500.00000,21104985000,21110074000,12654123000,211049850_211100740_126541230,51.674615,-3.212091
1210777,8355F009-8A15-55C5-E053-6B04A8C0D090,55,70000.0,FLINT,FLINTSHIRE,F,T,11/02/2019,CH6 5DY,1.0,0.000000,0.0,0.0,0.0,0.000000,14,CH6_02_2019,41333.33333,45416.66667,27804846000,6355589000,11793603000,278048460_63555890_117936030,53.245090,-3.123599
1210778,8A78B2B0-4326-5CB0-E053-6B04A8C0F504,81,95000.0,TODMORDEN,CALDERDALE,L,T,08/04/2019,OL14 8NL,1.0,88.443148,25.0,9.7,7.7,2.462274,16,OL14_04_2019,54000.00000,56650.00000,28051597900,2906478000,14022715000,280515979_29064780_140227150,53.734079,-2.140287


In [42]:
#separate dataset which to be used for the predicted values analysis
data_for_prediction_analysis = data.groupby(['location_key'])['postcode'].first().reset_index()
data_for_prediction_analysis_1 = data.groupby(['location_key'])['district'].first().reset_index()
data_for_prediction_analysis=pd.merge(data_for_prediction_analysis, data_for_prediction_analysis_1, how="left", on="location_key")
data_for_prediction_analysis

,location_key,postcode,district
0,100000460_235813700_173697830,IP28 6QS,ST EDMUNDSBURY
1,100000900_306645770_193413260,PO2 9AE,PORTSMOUTH
2,100001709_306232250_192988720,PO2 8NT,PORTSMOUTH
3,100002450_241021470_178393000,IP33 3UF,ST EDMUNDSBURY
4,100002500_240933210_178314290,IP33 3UE,ST EDMUNDSBURY
...,...,...,...
563834,99996430_240804040_178195840,IP33 3UN,WEST SUFFOLK
563835,99996800_303056370_189735890,PO16 9PA,FAREHAM
563836,99998470_302856560_189531620,PO16 9UH,FAREHAM
563837,99998690_306575700_193341460,PO2 9BS,PORTSMOUTH


In [43]:
#remove columns which are no longer required
data.drop(['unique_id' ,'town',	'district', 'deed_date',	'postcode','postcode_area_m_y', 'location_key' ],  axis='columns', inplace=True)
data = data.loc[:, ['num_of_beds', 'estate_type', 'property_type','num_of_sec_sch','score_disadv','closest_sch_distance', 'closest_disadv_prct',	'closest_aver_attain_eng',	'closest_aver_attain_math','median_ppb_3mnths_ago','median_ppb_4mnths_ago','hist_mnth',	'distances_Lond',	'distances_Manch','distances_Birming','lat',	'long','this_price_per_bedroom' ,'compare_ppd_re_hist_median_3mnths', 'compare_ppd_re_hist_median_4mnths','price_paid']]
data

,num_of_beds,estate_type,property_type,num_of_sec_sch,score_disadv,closest_sch_distance,closest_disadv_prct,closest_aver_attain_eng,closest_aver_attain_math,median_ppb_3mnths_ago,median_ppb_4mnths_ago,hist_mnth,distances_Lond,distances_Manch,distances_Birming,lat,long,this_price_per_bedroom,compare_ppd_re_hist_median_3mnths,compare_ppd_re_hist_median_4mnths,price_paid
0,1,L,T,23.0,110.178111,0.671216,96.0,0.5,0.9,60000.000000,56187.500000,7,26510768000,718540000,11622296000,53.489825,-2.358860,100000.0,66.666667,77.975528,100000.0
1,1,L,S,24.0,78.275669,0.320619,15.0,10.8,9.4,79166.666667,79000.000000,23,26539970000,542647000,11902983000,53.528583,-2.270335,79500.0,0.421053,0.632911,79500.0
2,1,L,F,12.0,85.979811,0.584296,20.0,11.5,9.8,73416.666667,64341.666667,4,16016690000,25505656000,21724217000,52.637255,1.298314,128000.0,74.347333,98.937961,128000.0
3,1,L,F,54.0,92.304597,0.543902,36.0,11.4,9.6,285000.000000,326250.000000,26,770794900,26771245000,16701868000,51.446749,-0.137157,314000.0,10.175439,-3.754789,314000.0
4,1,L,F,3.0,89.636307,0.452391,13.0,9.7,8.3,101666.666667,101666.666667,14,6296533000,26014878000,18039516000,51.868112,0.544362,170000.0,67.213115,67.213115,170000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1182761,5,F,D,1.0,0.000000,0.000000,0.0,0.0,0.0,87250.000000,92500.000000,16,18126308000,28825475000,18214946000,50.896436,-2.576097,114000.0,30.659026,23.243243,570000.0
1182762,5,F,D,24.0,97.697793,1.639435,97.0,1.9,2.8,105000.000000,107500.000000,4,25424316000,711278000,10659618000,53.417210,-2.246727,178000.0,69.523810,65.581395,890000.0
1182763,5,F,S,6.0,193.121503,1.094252,26.0,8.7,8.3,89750.000000,92500.000000,9,8815989000,34831047000,24742056000,50.868112,0.567342,65600.0,-26.908078,-29.081081,328000.0
1182764,5,F,D,6.0,46.650826,2.156095,27.0,9.5,8.1,57750.000000,57750.000000,2,29718009000,3717254000,14712579000,53.731672,-2.624259,42800.0,-25.887446,-25.887446,214000.0


In [44]:
#separate the main dataset into (1).the dataset which will be used for training and testing and (2).validation dataset
#we would like to the validation dataset to contain the latest data (we would like to predictict the most recent prices - i.e. Feb 2020)
#(1) training and testing dataset(approx 97%) and  (2) validation dataset (approx 3%) 
train_and_test_data=data[(data['hist_mnth']!=26)]
train_and_test_data

,num_of_beds,estate_type,property_type,num_of_sec_sch,score_disadv,closest_sch_distance,closest_disadv_prct,closest_aver_attain_eng,closest_aver_attain_math,median_ppb_3mnths_ago,median_ppb_4mnths_ago,hist_mnth,distances_Lond,distances_Manch,distances_Birming,lat,long,this_price_per_bedroom,compare_ppd_re_hist_median_3mnths,compare_ppd_re_hist_median_4mnths,price_paid
0,1,L,T,23.0,110.178111,0.671216,96.0,0.5,0.9,60000.000000,56187.500000,7,26510768000,718540000,11622296000,53.489825,-2.358860,100000.0,66.666667,77.975528,100000.0
1,1,L,S,24.0,78.275669,0.320619,15.0,10.8,9.4,79166.666667,79000.000000,23,26539970000,542647000,11902983000,53.528583,-2.270335,79500.0,0.421053,0.632911,79500.0
2,1,L,F,12.0,85.979811,0.584296,20.0,11.5,9.8,73416.666667,64341.666667,4,16016690000,25505656000,21724217000,52.637255,1.298314,128000.0,74.347333,98.937961,128000.0
4,1,L,F,3.0,89.636307,0.452391,13.0,9.7,8.3,101666.666667,101666.666667,14,6296533000,26014878000,18039516000,51.868112,0.544362,170000.0,67.213115,67.213115,170000.0
5,1,F,S,6.0,55.701643,1.752959,14.0,11.9,11.0,136250.000000,168104.166667,18,4111570000,27468619000,16632627000,51.236727,-0.560636,655000.0,380.733945,289.639361,655000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1182761,5,F,D,1.0,0.000000,0.000000,0.0,0.0,0.0,87250.000000,92500.000000,16,18126308000,28825475000,18214946000,50.896436,-2.576097,114000.0,30.659026,23.243243,570000.0
1182762,5,F,D,24.0,97.697793,1.639435,97.0,1.9,2.8,105000.000000,107500.000000,4,25424316000,711278000,10659618000,53.417210,-2.246727,178000.0,69.523810,65.581395,890000.0
1182763,5,F,S,6.0,193.121503,1.094252,26.0,8.7,8.3,89750.000000,92500.000000,9,8815989000,34831047000,24742056000,50.868112,0.567342,65600.0,-26.908078,-29.081081,328000.0
1182764,5,F,D,6.0,46.650826,2.156095,27.0,9.5,8.1,57750.000000,57750.000000,2,29718009000,3717254000,14712579000,53.731672,-2.624259,42800.0,-25.887446,-25.887446,214000.0


In [45]:
validation_data=data[(data['hist_mnth']==26)]
validation_data

,num_of_beds,estate_type,property_type,num_of_sec_sch,score_disadv,closest_sch_distance,closest_disadv_prct,closest_aver_attain_eng,closest_aver_attain_math,median_ppb_3mnths_ago,median_ppb_4mnths_ago,hist_mnth,distances_Lond,distances_Manch,distances_Birming,lat,long,this_price_per_bedroom,compare_ppd_re_hist_median_3mnths,compare_ppd_re_hist_median_4mnths,price_paid
3,1,L,F,54.0,92.304597,0.543902,36.0,11.4,9.6,285000.000000,326250.000000,26,770794900,26771245000,16701868000,51.446749,-0.137157,314000.0,10.175439,-3.754789,314000.0
50,1,F,T,35.0,78.178630,0.086689,36.0,8.9,8.4,113333.333333,119166.666667,26,2095469000,26684681000,17218624000,51.564821,0.123801,225000.0,98.529412,88.811189,225000.0
77,1,L,F,11.0,67.921534,0.836103,32.0,7.6,7.0,92750.000000,80000.000000,26,10333301000,30370523000,19027695000,50.830953,-1.173732,120000.0,29.380054,50.000000,120000.0
109,1,L,F,60.0,104.272807,0.530036,60.0,9.2,8.7,219083.333333,240000.000000,26,502176000,26527685000,16536000000,51.478533,-0.121591,500000.0,128.223659,108.333333,500000.0
200,1,L,F,9.0,125.415966,0.658497,39.0,9.0,7.6,42000.000000,42908.333333,26,32407942000,6604266000,17029935000,53.856740,-3.027827,52000.0,23.809524,21.188580,52000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1182652,5,F,D,6.0,10.055754,4.727453,9.0,9.8,8.5,52500.000000,55333.333333,26,22886393000,4770220000,7212565000,53.060015,-2.388207,78750.0,50.000000,42.319277,393750.0
1182676,5,F,S,1.0,62.208157,4.362943,38.0,7.5,7.3,92333.333333,97000.000000,26,8821511000,25299709000,18394460000,52.085717,0.720703,186000.0,101.444043,91.752577,930000.0
1182718,5,F,D,1.0,46.891396,2.650615,14.0,9.6,8.8,105520.833333,106666.666667,26,6938065000,27433863000,19483321000,51.800099,0.725616,112000.0,6.140178,5.000000,560000.0
1182725,5,F,T,9.0,103.448709,1.487324,16.0,10.7,10.2,163958.333333,158791.666667,26,7548684000,32765304000,21992537000,50.834805,-0.137428,108400.0,-33.885642,-31.734453,542000.0


In [46]:
from google.colab import files
with open('test data predictions.csv', 'w') as f:
  #f.write('some content')
  train_and_test_data.to_csv('train_and_test_data.csv', index=False)
  validation_data.to_csv('validation_data.csv', index=False)
  data_for_prediction_analysis.to_csv('locations_categorical_features.csv', index=False)
files.download('train_and_test_data.csv')    
files.download('validation_data.csv')  
files.download('locations_categorical_features.csv')  


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>